In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 原始数据
data = {
    'Model': ['Chai-1', 'Boltz-1', 'Protenix', 'Protenix+', 'Protenix-Mini', 'Protenix-Tiny', 'Protenix-Mini+', 'Protenix-Mini+|10,2'],
    'N cycle': [10, 10, 10, 4, 4, 4, 4, 10],
    'Diff Step': [200, 200, 200, 2, 2, 2, 2, 2],
    'Intra-Prot LDDT': [0.837, 0.840, 0.830, 0.815, 0.828, 0.797, 0.766, 0.801],
    'Prot-Prot LDDT': [0.500, 0.500, 0.516, 0.454, 0.465, 0.404, 0.369, 0.411],
    'Prot-Lig LDDT': [0.444, 0.493, 0.510, 0.465, 0.493, 0.448, 0.422, 0.451],
    'Prot-RNA LDDT': [0.153, 0.176, 0.178, 0.144, 0.139, 0.138, 0.112, 0.138],
    'Prot-DNA LDDT': [0.258, 0.306, 0.281, 0.296, 0.270, 0.272, 0.240, 0.275],
    'Ligand SR': [0.692, 0.724, 0.795, 0.749, 0.727, 0.705, 0.675, 0.711]
}
df = pd.DataFrame(data)

# 清理和统一模型名称
# 将最后一个特殊命名的模型标记为独特的名称以区分颜色和图例
df.loc[df.index[-1], 'Model'] = 'Protenix-Mini+ (10 cycle)'
df['Model'] = df['Model'].replace({'Protenix-Mini+': 'Protenix-Mini+ (4 cycle)'})


# 自定义调色板（包含 Chai-1 和 Boltz-1 的默认色，以及区分 Protenix-Mini+ 的颜色）
PALETTE = {
    "Chai-1":                     "#696969",  
    "Boltz-1":                    "#A9A9A9",  
    "Protenix":                   "#1A535C",  # deep teal
    "Protenix+":                  "#145A7A",  # darker teal
    "Protenix-Mini":              "#00C5CD",  # bright cyan-teal
    "Protenix-Mini+ (4 cycle)":   "#B3D9FF",  # light blue (n_cycle=4)
    "Protenix-Mini+ (10 cycle)":  "#8DE1C4",  # slightly darker blue (n_cycle=10)
    "Protenix-Tiny":              "#E0FFFF",  # very light cyan (pale aqua)
}

# 确保所有模型都在图例中，并保持正确的绘图顺序
model_names = df['Model'].unique().tolist()
model_colors = [PALETTE[name] for name in model_names]

# 需要绘制的指标列
metrics = ['Intra-Prot LDDT', 'Prot-Prot LDDT', 'Prot-Lig LDDT', 'Prot-RNA LDDT', 'Prot-DNA LDDT', 'Ligand SR']

# --- 绘图设置 ---
fig, ax = plt.subplots(figsize=(18, 8))

# 柱子的总数量 = 指标数量 * 模型数量
n_metrics = len(metrics)
n_models = len(df)

# 定义每个指标组的中心位置
metric_positions = np.arange(n_metrics)

# 单个柱子的宽度。我们希望 n_models 个柱子能容纳在一个单位宽度内，并留出间距
bar_width = 0.8 / n_models
# 柱子在组内的偏移量
offsets = np.linspace(-bar_width * (n_models - 1) / 2, 
                      bar_width * (n_models - 1) / 2, 
                      n_models)

# 循环绘制，为每个模型绘制一组柱子
for i, model_name in enumerate(model_names):
    # 提取当前模型在所有指标上的数据
    model_data = df.loc[df['Model'] == model_name, metrics].values.flatten()
    
    # 计算当前模型柱子的 x 轴位置
    x_positions = metric_positions + offsets[i]
    
    # 绘制柱子
    bars = ax.bar(x_positions, model_data, 
                  bar_width, 
                  label=model_name, 
                  color=model_colors[i],
                  edgecolor='none',
                  linewidth=0.5)
    
    # 添加数值标签（仅添加关键模型的标签，否则图表会过于拥挤）
    # 这里我们只对 Protenix (n_cycle=10) 添加标签作为示例
    if model_name in model_names: # ['Protenix', 'Boltz-1', 'Protenix-Mini+ (10 cycle)']:
        for x_pos, height in zip(x_positions, model_data):
            ax.text(x_pos, height + 0.005, f'{height:.3f}', 
                    ha='center', va='bottom', fontsize=11, rotation=90, 
                    color='black')

# --- 格式化图表 ---

# 设置 X 轴刻度标签为指标名称
ax.set_xticks(metric_positions)
ax.set_xticklabels(metrics, fontsize=15)

# 设置 Y 轴标签
ax.set_ylabel('Performance Score (Higher is Better)', fontsize=15)

# 设置标题
# ax.set_title('Model Performance Across Six Different Interaction Prediction Tasks', fontsize=16)

# 调整 Y 轴范围以适配所有数据（所有值都在 0.0 到 0.9 之间）
ax.set_ylim(0, 0.9)
ax.set_xlim(-0.4, 5.5)
ax.grid(axis='y', linestyle='--', alpha=0.7)

# 添加图例
ax.legend(loc='upper center', 
          bbox_to_anchor=(0.5, 0.95), 
          ncol=4, 
          fontsize=18, 
          frameon=False)
ax.set_yticks([0.0, 0.4, 0.8])
ax.grid(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
# plt.show()
plt.savefig('protenix_mini_plus_main_results.pdf')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


data = {
    'Model Variant': [
        'Protenix-Mini+', 
        'w/o distill', 
        'Only linear attention', 
        'Only chunked update', 
        'All blocks with chunked update', 
        'Mini+ from scratch'
    ],
    'Complex LDDT': [0.796, 0.796, 0.800, 0.799, 0.744, 0.789],
    'Intra-Prot LDDT': [0.820, 0.819, 0.829, 0.830, 0.786, 0.818],
    'Prot-Prot LDDT': [0.423, 0.418, 0.438, 0.440, 0.354, 0.424],
    'Prot-Lig LDDT': [0.596, 0.616, 0.624, 0.611, 0.557, 0.604],
    'PoseBuster SR': [0.700, 0.698, 0.712, 0.691, 0.552, 0.692]
}
df = pd.DataFrame(data)

# **新的定制调色板**
PALETTE_VARIANTS = {
    "Protenix-Mini+":               "#1A535C",  # Deep Teal (Baseline)
    "w/o distill":                  "#2D7380",  # Darker Teal
    "Only linear attention":        "#4CA5B3",  # Bright Cyan-Teal
    "Only chunked update":          "#7BCDD9",  # Light Cyan
    "All blocks with chunked update": "#B3E3E9",  # Brick Red (Highlight Low Performance)
    "Mini+ from scratch":           "#E7F3F5",  # Light Blue
}

model_names = df['Model Variant'].tolist()
model_colors = [PALETTE_VARIANTS[name] for name in model_names]

# 需要绘制的指标列
metrics = ['Complex LDDT', 'Intra-Prot LDDT', 'Prot-Prot LDDT', 'Prot-Lig LDDT', 'PoseBuster SR']

# --- 绘图设置 ---
fig, ax = plt.subplots(figsize=(16, 7))

n_metrics = len(metrics)
n_models = len(df)

metric_positions = np.arange(n_metrics)
bar_width = 0.85 / n_models
offsets = np.linspace(-bar_width * (n_models - 1) / 2, 
                      bar_width * (n_models - 1) / 2, 
                      n_models)

# 循环绘制，为每个模型变体绘制一组柱子
for i, model_name in enumerate(model_names):
    model_data = df.loc[df['Model Variant'] == model_name, metrics].values.flatten()
    x_positions = metric_positions + offsets[i]
    
    # 绘制柱子 (边框已移除)
    ax.bar(x_positions, model_data, 
           bar_width, 
           label=model_name, 
           color=model_colors[i],
           edgecolor='none')
    
    if model_name in model_names:
        for x_pos, height in zip(x_positions, model_data):
            # 自动选择标签颜色以保证可见性
            # label_color = 'white' if model_name in ['Protenix-Mini+', 'All blocks with chunked update'] and height > 0.4 else 'black'
            ax.text(x_pos, height + 0.005, f'{height:.3f}', 
                    ha='center', va='bottom', fontsize=10, rotation=90, 
                    color='black')

# --- 格式化图表 ---

# 设置 X 轴刻度标签为指标名称
ax.set_xticks(metric_positions)
ax.set_xticklabels(metrics, fontsize=12)

# 设置 Y 轴标签
ax.set_ylabel('Performance Score', fontsize=14)

# 调整 Y 轴范围 (基于数据范围 0.354 到 0.830)
ax.set_ylim(0.3, 0.85) 
ax.set_xlim(-0.425, 4.45) 

# 添加图例
ax.legend(loc='upper center', 
          bbox_to_anchor=(0.5, 1.15), 
          ncol=6, 
          fontsize=12., 
          frameon=False)

# 清理边框和网格线
# ax.grid(axis='y', linestyle='--', alpha=0.5)
ax.grid(False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False) # 移除左侧边框以达到更简洁的效果
ax.tick_params(axis='y', length=0) # 移除Y轴刻度线

plt.tight_layout()
# plt.show()

# 如果需要保存为PDF，取消注释下一行
plt.savefig('protenix_mini_plus_variants.pdf')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 原始数据 (来自截屏2025-10-14 下午2.15.08.png)
data = {
    'Index': ['Protenix_10_200', 'Protenix-Mini_10_200', 'Protenix-Mini_10_2', 'Protenix-Mini_4_2', 'Protenix-Mini+_10_2', 'Protenix-Mini+_4_2'],
    'Model': ['Protenix', 'Protenix-Mini', 'Protenix-Mini', 'Protenix-Mini', 'Protenix-Mini+', 'Protenix-Mini+'],
    'Diff Step': [200, 200, 2, 2, 2, 2],
    'N cycle': [10, 10, 10, 4, 10, 4],
    'DockQ SR': [0.185, 0.163, 0.163, 0.141, 0.127, 0.096],
    'iLDDT': [0.169, 0.146, 0.143, 0.114, 0.122, 0.109],
}
df = pd.DataFrame(data)

PALETTE_VARIANTS = {
    "Protenix_10_200":               "#1A535C",  # Deep Teal (Baseline)
    "Protenix-Mini_10_200":                  "#145A7A",  # Darker Teal
    "Protenix-Mini_10_2":        "#00C5CD",  # Bright Cyan-Teal
    "Protenix-Mini_4_2":          "#B3D9FF",  # Light Cyan
    "Protenix-Mini+_10_2": "#8DE1C4",  # Brick Red (Highlight Low Performance)
    "Protenix-Mini+_4_2":           "#E0FFFF",  # Light Blue
}

model_names = df['Index'].tolist()
model_colors = [PALETTE_VARIANTS[name] for name in model_names]

# 需要绘制的指标列
metrics = ['Diff Step', 'N cycle', 'DockQ SR', 'iLDDT']

# --- 绘图设置 ---
fig, ax = plt.subplots(figsize=(16, 7))

n_metrics = len(metrics)
n_models = len(df)

metric_positions = np.arange(n_metrics)
bar_width = 0.85 / n_models
offsets = np.linspace(-bar_width * (n_models - 1) / 2, 
                      bar_width * (n_models - 1) / 2, 
                      n_models)

# 循环绘制，为每个模型变体绘制一组柱子
for i, model_name in enumerate(model_names):
    model_data = df.loc[df['Index'] == model_name, metrics].values.flatten()
    x_positions = metric_positions + offsets[i]

    # 绘制柱子 (边框已移除)
    ax.bar(x_positions, model_data / np.array([400, 20, 1., 1.]), 
           bar_width, 
           label=model_name.split('_')[0], 
           color=model_colors[i],
           edgecolor='none')
    
    if model_name in model_names:
        idx = 0
        for x_pos, height in zip(x_positions, model_data):
            if idx == 0:
                norm = 400
            elif idx == 1:
                norm = 20
            else:
                norm = 1

            if idx < 2:
                ax.text(x_pos, (height / norm + 0.002), f'{int(height):d}', 
                    ha='center', va='bottom', fontsize=14, rotation=90, 
                    color='black')
            else:
                ax.text(x_pos, (height / norm + 0.002), f'{height:.3f}', 
                    ha='center', va='bottom', fontsize=14, rotation=90, 
                    color='black')
            idx += 1

# --- 格式化图表 ---

# 设置 X 轴刻度标签为指标名称
ax.set_xticks(metric_positions)
ax.set_xticklabels(metrics, fontsize=12)

# 设置 Y 轴标签
# ax.set_ylabel('Performance Score', fontsize=14)

ax.set_yticks([0, 1], ["", ""])
ax.set_ylim(0., .5) 
ax.set_xlim(-0.425, 3.45) 

# 添加图例
ax.legend(loc='upper center', 
          bbox_to_anchor=(0.5, 1.15), 
          ncol=6, 
          fontsize=12., 
          frameon=False)

# 清理边框和网格线
# ax.grid(axis='y', linestyle='--', alpha=0.5)
ax.grid(False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False) # 移除左侧边框以达到更简洁的效果
ax.tick_params(axis='y', length=0) # 移除Y轴刻度线

plt.tight_layout()
# plt.show()
plt.savefig('protenix_mini_plus_prot_prot_interface.pdf')


In [ ]:
###FLOPS to seq lens###

import os
import numpy as np
import matplotlib.pyplot as plt
# --- Font setup (clean, non-bold like your reference figure) ---

plt.rcParams.update({
    "font.size": 14,          # default font size
    "axes.titlesize": 16,     # title
    "axes.labelsize": 14,     # x/y labels
    "xtick.labelsize": 12,    # tick labels
    "ytick.labelsize": 12,    
    "legend.fontsize": 12,    # legend text
    "font.family": "sans-serif",
    "font.sans-serif": ["DejaVu Sans"],  # matches your figure
})

os.makedirs("figs", exist_ok=True)

# Define n range
n = [1, 5, 10, 50]

# Define complexity functions (operations count)
f1 = [0.169, 0.192, 0.201, 0.231]
f2 = [0.122, 0.162, 0.184, 0.218]
f3 = [0.109, 0.152, 0.163, 0.183]

# Convert to TFLOPs (1e12 operations = 1 TFLOP)
f1_tflops = f1 
f2_tflops = f2 
f3_tflops = f3 

# Colors: Pairformer, Pairformer+chunk (turquoise), Pairformer+chunk+linear (lighter blue)
palette = ["#1A535C", "#00C5CD", "#B3D9FF"]
labels = [
    r"Protenix",
    r"Protenix+",
    r"Protenix-Mini+",
]

# --- Main figure (no legend) ---
fig, ax = plt.subplots(figsize=(8, 4.5), dpi=300)
ax.plot(n, f1_tflops, label=labels[0], color=palette[0], lw=2.2)
ax.plot(n, f2_tflops, label=labels[1], color=palette[1], lw=2.2)
ax.plot(n, f3_tflops, label=labels[2], color=palette[2], lw=2.2)

ax.set_xlabel(r"$n_{\mathrm{Seed}}$", fontsize=16)
ax.set_ylabel("iLDDT", fontsize=16)
ax.tick_params(axis="both", which="major", labelsize=12)

# Clean frame: remove grid + top/right spines
ax.grid(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.legend(loc='upper center', 
          bbox_to_anchor=(0.5, 1.15), 
          ncol=3, 
          fontsize=12., 
          frameon=False)
# Save main plot
handles, legend_labels = ax.get_legend_handles_labels()
ax.set_xlim(1, 51)
ax.set_xticks([1, 5, 10, 50])
fig.tight_layout()

plt.savefig('protenix_mini_plus_ab_interface_seed_scaling.pdf')

